<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/pytorch/t81_558_class_03_3_save_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
**Module 3: Introduction to PyTorch**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 3 Material

* Part 3.1: Deep Learning and Neural Network Introduction [[Video]](https://www.youtube.com/watch?v=OaJntP14cRA&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_1_neural_net.ipynb)
* Part 3.2: Introduction to PyTorch [[Video]](https://www.youtube.com/watch?v=z5X2qV5h_p0&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_2_pytorch.ipynb)
* **Part 3.3: Saving and Loading a PyTorch Neural Network** [[Video]](https://www.youtube.com/watch?v=NkG8w_Ua2Yo&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_3_save_load.ipynb)
* Part 3.4: Early Stopping in PyTorch to Prevent Overfitting [[Video]](https://www.youtube.com/watch?v=7Fboe7_aTtY&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_4_early_stop.ipynb)
* Part 3.5: Extracting Weights and Manual Calculation [[Video]](https://www.youtube.com/watch?v=Fw9VqcqFP_c&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_03_5_weights.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed. Running the following code will map your GDrive to ```/content/drive```. We also initialize the PyTorch device to either GPU/MPS (if available) or CPU.

In [1]:
import torch

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available. (see module 3.2)
device = "mps" if getattr(torch,'has_mps',False) \
    else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Mounted at /content/drive
Note: using Google CoLab
Using device: cuda


# Part 3.3: Saving and Loading a PyTorch Neural Network

Complex neural networks will take a long time to fit/train. It is helpful to be able to save these neural networks so that you can reload them later. A reloaded neural network will not require retraining. PyTorch usually saves neural networks as [pickle](https://wiki.python.org/moin/UsingPickle) files. The following code trains a neural network to predict car MPG and saves the model.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
from sklearn import preprocessing

# You will create a network class for every PyTorch neural network you create.
class Net(nn.Module):
    def __init__(self, in_count, out_count):
        super(Net, self).__init__()
        # We must define each of the layers.
        self.fc1 = nn.Linear(in_count, 50)
        self.fc2 = nn.Linear(50, 25)
        self.fc3 = nn.Linear(25, 1)

    def forward(self, x):
        # In the forward pass, we must calculate all of the layers we 
        # previously defined.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Read the MPG dataset.
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Numpy to PyTorch
x = torch.tensor(x,device=device,dtype=torch.float32)
y = torch.tensor(y,device=device,dtype=torch.float32)


# Define the neural network
model = Net(x.shape[1],1).to(device)

# Define the loss function for regression
loss_fn = nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train for 1000 epochs.
model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(x).flatten()
    loss = loss_fn(out, y)
    loss.backward()
    optimizer.step()
    
    # Display status every 100 epochs.
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, loss: {loss.item()}")

model.eval()
pred = model(x)

# Measure RMSE error.  RMSE is common for regression.
score = torch.sqrt(torch.nn.functional.mse_loss(pred.flatten(),y))
print(f"Before save score (RMSE): {score}")
torch.save(model, "mpg.pkl")

Epoch 0, loss: 12226.310546875
Epoch 100, loss: 136.61398315429688
Epoch 200, loss: 50.00810241699219
Epoch 300, loss: 25.583969116210938
Epoch 400, loss: 18.813331604003906
Epoch 500, loss: 18.46914291381836
Epoch 600, loss: 14.9017915725708
Epoch 700, loss: 12.732271194458008
Epoch 800, loss: 11.355725288391113
Epoch 900, loss: 10.49295425415039
Before save score (RMSE): 3.160562515258789


The code below sets up a neural network and reads the data (for predictions), but it does not clear the model directory or fit the neural network. The code loads the weights from the previous fit. Now we reload the network and perform another prediction. The RMSE should match the previous one exactly if we saved and reloaded the neural network correctly.

In [3]:
model2 = torch.load("mpg.pkl")
pred = model2(x)
# Measure RMSE error.  RMSE is common for regression.
score = torch.sqrt(torch.nn.functional.mse_loss(pred.flatten(),y))
print(f"After load score (RMSE): {score}")

After load score (RMSE): 3.160562515258789
